In [ ]:
#https://www.tensorflow.org/probability/examples/Probabilistic_Layers_VAE#dependencies_prerequisites
import keras
from tensorflow.keras import layers
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K
#import tensorflow_addons as tfa

input_shape = (50, 50, 3)
num_features = 7500#50*50*3
latent_dim = 32

In [ ]:
def encoder_CNN(input_shape = (50, 50, 3), latent_dim = 2):

    inputs = layers.Input(shape=input_shape, name='Input')

    # layer 1
    x = layers.Conv2D(filters=6, kernel_size=5, strides=1,padding='valid',name='C1')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
   # x = layers.MaxPool2D(pool_size=2, strides=2,name='S2')(x)

    # layer 2
    x = layers.Conv2D(filters=16, kernel_size=5,strides=1,padding='valid',name='C3')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
   # x = layers.MaxPool2D(pool_size=2, strides=2,name='S4')(x)

    # layer 3
    x = layers.Conv2D(filters=120, kernel_size=5,strides=1,padding='valid',name='C5')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Flatten()(x)

    y = layers.Dense(84, activation ='relu',name='F6')(x)
    outputs = layers.Dense(latent_dim, activation='relu', name='outputs')(y)
    model = keras.Model(inputs, outputs, name='encoder')
    
    return model

In [16]:
def encoder_CNN(input_shape = (50, 50, 3), latent_dim = 2):

    inputs = layers.Input(shape=input_shape, name='Input')

    # layer 1
    x = layers.Conv2D(filters=64, kernel_size=2, strides=1,padding='valid',name='C1')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
   # x = layers.MaxPool2D(pool_size=2, strides=2,name='S2')(x)

    # layer 2
    x = layers.Conv2D(filters=64, kernel_size=2,strides=1,padding='valid',name='C3')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
   # x = layers.MaxPool2D(pool_size=2, strides=2,name='S4')(x)

    # layer 3
    x = layers.Conv2D(filters=64, kernel_size=3,strides=1,padding='valid',name='C5')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)


    x = layers.Flatten()(x)

    y = layers.Dense(256, activation ='relu',name='F6')(x)
    
    outputs = layers.Dense(latent_dim, activation='relu', name='outputs')(y)
    model = keras.Model(inputs, outputs, name='encoder')
    
    return model

In [26]:
encoder = encoder_CNN(latent_dim = latent_dim)
encoder.summary()


Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 50, 50, 3)]       0         
                                                                 
 conv_block_12 (ConvBlock)   (None, 49, 49, 64)        1088      
                                                                 
 conv_block_13 (ConvBlock)   (None, 48, 48, 64)        16704     
                                                                 
 conv_block_14 (ConvBlock)   (None, 46, 46, 64)        37184     
                                                                 
 F6 (Dense)                  (None, 46, 46, 256)       16640     
                                                                 
 outputs (Dense)             (None, 46, 46, 100)       25700     
                                                                 
Total params: 97,316
Trainable params: 96,932
Non-trainable

In [ ]:
def decoder_CNN(input_shape = (50, 50, 3), latent_dim = 2):
    latent_inputs = layers.Input(shape=(latent_dim,))
    x = layers.Dense(84, activation ='tanh',name='F6l')(latent_inputs)
    x = layers.Dropout(0.3)(latent_inputs)
    x = layers.Dense(5*5*120, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Reshape(target_shape=(5, 5, 120))(x) #(5, 5, 120)
    #x = tfa.layers.MaxUnpooling2D((2,2), name='S4l')(x) # (10, 10, 120)
    x = layers.Conv2DTranspose(filters=16, kernel_size=5,strides=1,padding='valid',name='C5l')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
   # x = tfa.layers.MaxUnpooling2D((2,2))(x)
    x = layers.Conv2DTranspose(filters=6, kernel_size=5, strides=1,padding='valid',name='C1l')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    outputs = layers.Conv2DTranspose(filters=3, kernel_size=19, strides=1,activation='sigmoid',padding='valid')(x)
   # outputs = layers.Reshape(target_shape=(50, 50, 3), name='output')(x)
    model = keras.Model(latent_inputs, outputs, name='decoder')
    return model

    

In [ ]:
def decoder_CNN(input_shape = (50, 50, 3), latent_dim = 2):
    latent_inputs = layers.Input(shape=(latent_dim,))

    #layer 1
    x = layers.Dense(256, activation ='tanh',name='F6l')(latent_inputs)

    #layer 2
    x = layers.Dense(25*25*64, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Reshape((25, 25, 64))(x)

    #x = layers.Reshape(target_shape=(5, 5, 120))(x) #(5, 5, 120)
    # layer 3
    x = layers.Conv2DTranspose(filters=64, kernel_size=3,strides=1,padding='valid',name='C5l')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    # layer 4
    x = layers.Conv2DTranspose(filters=64, kernel_size=3, strides=2,padding='valid',name='C1l')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)


    outputs = layers.Conv2D(filters=3, kernel_size=6, strides=1,activation='sigmoid',padding='valid')(x)

    model = keras.Model(latent_inputs, outputs, name='decoder')
    return model


In [ ]:
decoder = decoder_CNN(latent_dim = latent_dim)
decoder.summary()

In [ ]:
def AE(input_shape, latent_dim, encoder, decoder):
    vae_input = layers.Input(shape = input_shape, name="AE_input")
    encoder_output = encoder(vae_input)
    decoder_output = decoder(encoder_output)
    model = keras.Model(vae_input, decoder_output, name='AE')
    return model

In [ ]:
ae = AE(input_shape, latent_dim, encoder, decoder)
ae.summary()
plot_model(ae,show_shapes=True, show_layer_names=True,expand_nested=True)

In [ ]:
def ae_loss(y_true, y_predict):
    reconstruction_loss_factor = 1
    reconstruction_loss = K.mean(K.square(y_true - y_predict), axis=[1, 2, 3])
    return reconstruction_loss_factor * reconstruction_loss

In [ ]:
#Information needed to compute the loss function
ae_input = ae.input
ae_output = ae.output

ae.add_loss(ae_loss( ae_input, ae_output))


In [ ]:
ae.compile(optimizer='adam')
ae.save('models/ae.h5')  
encoder.save('models/ae_encoder.h5')
decoder.save('models/ae_decoder.h5')    
ae.compile(optimizer='adam')